# Experiment: C3 - Mondial - Foreign Key - GPT 3.5

In [1]:

from langchain.chat_models import ChatOpenAI
from urllib.parse import quote  

from dotenv import load_dotenv
import time
import os
import sys
import json
load_dotenv()

from c3_clear_prompting import generate_clear_prompting
from c3_calibration_with_hints import generate_calibration_with_hints
from c3_generate_sql import generate_sql

experiment_path = '..\..'
path = os.path.abspath('')
module_path = os.path.join(path, experiment_path)
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from sqldatabase_langchain_utils import SQLDatabaseLangchainUtils

<>:15: SyntaxWarning: invalid escape sequence '\.'
<>:15: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Tahsin\AppData\Local\Temp\ipykernel_27000\2972517568.py:15: SyntaxWarning: invalid escape sequence '\.'
  experiment_path = '..\..'


c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\C3\..\..


In [2]:
SCHEMA = 'mondial_gpt'
PREFIX = 'mondial'
FILE_NAME_RESULT = f"results/5_c3_queries_chatgpt_{SCHEMA}_fk.json"

In [3]:
def save_queries(queries):
    data = {"queries":queries}
    with open(FILE_NAME_RESULT, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 
        
def read_queries():
    with open(FILE_NAME_RESULT, encoding='utf-8', errors='ignore') as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries


## Connection with the bank

In [4]:
json_file_path = f"{experiment_path}/datasets/{SCHEMA}_db_connection.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    db_connection = json.load(json_data, strict=False)
db_connection

{'DB_HOST': 'localhost',
 'DB_PORT': '1522',
 'DB_USER_NAME': 'MONDIAL_GPT',
 'DB_PASS': 'TextDB123',
 'DB_NAME': '',
 'SQL_DRIVER': 'oracle+oracledb',
 'SERVICE_NAME': 'xepdb1',
 'SCHEMA': 'MONDIAL_GPT',
 'KEYWORD_SEARCH_API_URL': ''}

In [5]:
db = SQLDatabaseLangchainUtils(db_connection=db_connection)

exclusao = [
    f"{SCHEMA}_tmdp",
    f"{SCHEMA}_tmdpmap",
    f"{SCHEMA}_tmds",
    f"{SCHEMA}_tmjmap",
    f"{SCHEMA}_tpv",
    f"{SCHEMA}_tmdc",
    f"{SCHEMA}_tmdcmap",
    f"{SCHEMA}_tmdej",
    f"{SCHEMA}_log_action",
    f"{SCHEMA}_log_error",
    f"{SCHEMA}_favorite_item", 
    f"{SCHEMA}_favorite_query",
    f"{SCHEMA}_favorite_tag",
    f"{SCHEMA}_favorite_tag_item",
    f"{SCHEMA}_favorite_visualization",
    f"{SCHEMA}_dashboard",
    f"{SCHEMA}_history",
    "teste_cliente",
    "teste_fornecedor",
    "teste_funcionario"
]

include_tables = [s for s in db.get_table_names() if not s.startswith(PREFIX) and s not in exclusao]
db = SQLDatabaseLangchainUtils(db_connection=db_connection, include_tables=include_tables, schema=db_connection.get("SCHEMA", None))
db.get_table_names()

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'coordinates'
  self._metadata.reflect(
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'source'
  self._metadata.reflect(
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\venv\Lib\site-packages\langchain_community\utilities\sql_database.py:123: SAWarning: Did not recognize type 'GEOCOORD' of column 'estuary'
  self._metadata.reflect(


['airport',
 'borders',
 'city',
 'citylocalname',
 'cityothername',
 'citypops',
 'continent',
 'country',
 'countrylocalname',
 'countryothername',
 'countrypops',
 'desert',
 'economy',
 'encompasses',
 'ethnicgroup',
 'geo_desert',
 'geo_estuary',
 'geo_island',
 'geo_lake',
 'geo_mountain',
 'geo_river',
 'geo_sea',
 'geo_source',
 'island',
 'islandin',
 'ismember',
 'lake',
 'lakeonisland',
 'language',
 'located',
 'locatedon',
 'mergeswith',
 'mountain',
 'mountainonisland',
 'organization',
 'politics',
 'population',
 'province',
 'provincelocalname',
 'provinceothername',
 'provpops',
 'religion',
 'river',
 'riveronisland',
 'riverthrough',
 'sea']

## C3 - Function

In [6]:
def run_c3(question, db, model='gpt-3.5-turbo', add_fk = True, callback= None):
    llm = ChatOpenAI(model_name = model, temperature=0.7, n=10)
    clear_prompting = generate_clear_prompting(question, db, llm, add_fk=add_fk, callback=callback)
    print(clear_prompting)
    messages = generate_calibration_with_hints(clear_prompting)
    llm = ChatOpenAI(model_name = model, n=20)
    sql = generate_sql(messages, llm, db, question, callback=callback)
    return sql

## Preparing consultations

In [7]:

json_file_path = f"../../datasets/{PREFIX}/queries_{PREFIX}.json"
with open(json_file_path, encoding='utf-8', errors='ignore') as json_data:
    queries = json.load(json_data, strict=False)
queries = queries['queries']
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': '',
  'type': 'simple'},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': '',
  'type': 'simple'},
 {'id': '3',
  'question': 'What are the languages spoken in Poland?',
  'query_string': '',
  'type': 'medium'},
 {'id': '4',
  'question': 'How deep is Lake Kariba?',
  'query_string': '',
  'type': 'simple'},
 {'id': '5',
  'question': 'What is the total of provinces of Netherlands?',
  'query_string': '',
  'type': 'complex'},
 {'id': '6',
  'question': 'What is the percentage of religious people are hindu in thailand?',
  'query_string': '',
  'type': 'complex'},
 {'id': '7',
  'question': 'List the number of provinces each river flows through.',
  'query_string': '',
  'type': 'medium'},
 {'id': '8',
  'question': 'Find all countries that became independent between 8/1/1910 and 8/1/1950.',
  'query_string': '',
  'type': 'complex'},
 {'id': '9',
  'que

## Tracking token usage

In [8]:
track_token = [] 
def tracking_token(cb =None, reset = False):
    global track_token
    track_token.append(cb)
    if reset:
        track_token = []

In [9]:

def convert_to_dict_tracking_token():
    token_usage = {}
    for e in track_token:
        for key in e.keys():
            token_usage[key] = {}
            token_usage[key]['total_tokens'] = e[key].total_tokens
            token_usage[key]['total_cost'] = e[key].total_cost
            token_usage[key]['prompt_tokens'] = e[key].prompt_tokens
            token_usage[key]['completion_tokens'] = e[key].completion_tokens 
    return token_usage

In [10]:
sql = run_c3("'What are the languages spoken in Poland?", db, callback=tracking_token)
print(convert_to_dict_tracking_token())

C:\Users\Tahsin\AppData\Local\Temp\ipykernel_27000\1352675344.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name = model, temperature=0.7, n=10)
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\C3\..\..\functions\sqldatabase_langchain_utils.py:98: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "mondial_gpt.lake, mondial_gpt.river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  return self.db._metadata.sorted_tables
c:\Users\Tahsin\Desktop\Uni\COSC\Di

Column recall attempt: 1

### Complete oracle SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query. 
### Oracle SQL tables, with their properties: 
#
# country (code, name, capital, province, population)
# countryothername (country, othername)
# language (name, country, percentage)
# city (name, country, province, population)
# countryothername.country=country.code
# language.country=country.code
# city.country=country.code
# city.province=province.name
# city.country=country.code
# city.province=province.name

#
### 'What are the languages spoken in Poland?
SELECT
{'table_recall': {'total_tokens': 2820, 'total_cost': 0.0052025000000000005, 'prompt_tokens': 875, 'completion_tokens': 1945}, 'column_recall': {'total_tokens': 3335, 'total_cost': 0.0065095, 'prompt_tokens': 321, 'completion_tokens': 3014}, 'sql_generation': {'total_tokens': 885, 'total_cost': 0.001486, 'prompt_tokens': 568, 'completion_tokens': 317}}


In [11]:
print(sql)
track_token


SELECT ```sql SELECT name FROM language WHERE country = 'POL' ```


[{'table_recall': Tokens Used: 2820
  	Prompt Tokens: 875
  		Prompt Tokens Cached: 0
  	Completion Tokens: 1945
  		Reasoning Tokens: 0
  Successful Requests: 1
  Total Cost (USD): $0.0052025000000000005},
 {'column_recall': Tokens Used: 3335
  	Prompt Tokens: 321
  		Prompt Tokens Cached: 0
  	Completion Tokens: 3014
  		Reasoning Tokens: 0
  Successful Requests: 1
  Total Cost (USD): $0.0065095},
 {'sql_generation': Tokens Used: 885
  	Prompt Tokens: 568
  		Prompt Tokens Cached: 0
  	Completion Tokens: 317
  		Reasoning Tokens: 0
  Successful Requests: 1
  Total Cost (USD): $0.001486}]

## Running the method

In [12]:
tracking_token(reset=True)
errors = []
query_index = -1
for instance in queries:
    query_index +=1
    try:
        start_time = time.time()
        sql = run_c3(instance["question"], db, callback=tracking_token)
        end_time = time.time()
        instance["query_string"] = sql
        instance["token_usage"] = convert_to_dict_tracking_token()
        instance['time'] = end_time - start_time
        save_queries(queries)
        print(instance['id'], instance['question'], instance["query_string"], instance['time'])
        print("-----")
    except:
        print("Error")
        errors.append(query_index)
        pass
    finally:
        tracking_token(reset=True)




c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\C3\..\..\functions\sqldatabase_langchain_utils.py:98: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "mondial_gpt.lake, mondial_gpt.river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  return self.db._metadata.sorted_tables


list error
list error


c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\C3\..\..\functions\sqldatabase_langchain_utils.py:98: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "mondial_gpt.lake, mondial_gpt.river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  return self.db._metadata.sorted_tables


Column recall attempt: 1

### Complete oracle SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query. 
### Oracle SQL tables, with their properties: 
#
# country (name, area, code, capital, population)
# province (name, country, area, population, capital)
# city (country, province, population, name, latitude)
# provinceothername (province, country, othername)
# province.country=country.code
# province.country=country.code
# city.country=country.code
# city.province=province.name
# provinceothername.province=province.name
# provinceothername.country=country.code
# city.country=country.code
# city.province=province.name

#
### What is the area of Thailand?
SELECT
1 What is the area of Thailand? SELECT area FROM country WHERE name = 'Thailand'; 30.162826776504517
-----


c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\C3\..\..\functions\sqldatabase_langchain_utils.py:98: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "mondial_gpt.lake, mondial_gpt.river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  return self.db._metadata.sorted_tables
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\C3\..\..\functions\sqldatabase_langchain_utils.py:98: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "mondial_gpt.lake, mondial_gpt.river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  return self.db._metadata.sorted_tables


...

c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\C3\..\..\functions\sqldatabase_langchain_utils.py:98: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "mondial_gpt.lake, mondial_gpt.river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  return self.db._metadata.sorted_tables
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\C3\..\..\functions\sqldatabase_langchain_utils.py:98: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "mondial_gpt.lake, mondial_gpt.river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  return self.db._metadata.sorted_tables


Column recall attempt: 1

### Complete oracle SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query. 
### Oracle SQL tables, with their properties: 
#
# country (code, name, capital, province)
# politics (country, independence, wasdependent, dependent, government)
# politics.country=country.code

#
### What type of government is Iran?
SELECT
99 What type of government is Iran? SELECT ```sql government FROM politics WHERE country = 'IRN'; ``` 10.009397029876709
-----


c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\C3\..\..\functions\sqldatabase_langchain_utils.py:98: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "mondial_gpt.lake, mondial_gpt.river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  return self.db._metadata.sorted_tables
c:\Users\Tahsin\Desktop\Uni\COSC\Directed Studies\TextToSQL\experiments\C3\..\..\functions\sqldatabase_langchain_utils.py:98: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "mondial_gpt.lake, mondial_gpt.river", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  return self.db._metadata.sorted_tables


Column recall attempt: 1

### Complete oracle SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query. 
### Oracle SQL tables, with their properties: 
#
# country (name, code, capital, province, area)
# economy (country, gdp, agriculture, industry, service)
# economy.country=country.code

#
### What is the percentage of industries in relation to Japan's economy?
SELECT
100 What is the percentage of industries in relation to Japan's economy? SELECT ```sql SELECT (industry / (agriculture + industry + service)) * 100 AS industry_percentage FROM economy WHERE country = 'JPN'; ``` 6.181938648223877
-----


: 

NameError: name 'errors' is not defined

In [ ]:
errors

#### Fixing query

In [12]:
pos = 40
instance = queries[pos]
q = read_queries()
start_time = time.time()
sql = run_c3(instance["question"], db, callback=tracking_token)
end_time = time.time()
instance["query_string"] = sql
instance["token_usage"] = convert_to_dict_tracking_token()
instance['time'] = end_time - start_time
q[pos] = instance
save_queries(q)
print(instance['id'], instance['question'], instance["query_string"], instance['time'])



Column recall attempt: 1

### Complete oracle SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query. 
### Oracle SQL tables, with their properties: 
#
# country (name, code, capital, province)
# sea (name, area, depth)
# borders (country1, country2, length)
# borders.country1=country.code
# borders.country2=country.code

#
### How many countries that are close to the Mediterranean Sea?
SELECT
41 How many countries that are close to the Mediterranean Sea? SELECT COUNT(DISTINCT country.name) FROM country JOIN borders ON country.code = borders.country1 JOIN sea ON borders.country2 = sea.name WHERE sea.name = 'Mediterranean Sea'; 42.66082406044006


In [ ]:
to_fix = [59,62,72,85,99]
for pos in to_fix:
    instance = queries[pos]
    q = read_queries()
    start_time = time.time()
    sql = run_c3(instance["question"], db, callback=tracking_token)
    end_time = time.time()
    instance["query_string"] = sql
    instance["token_usage"] = convert_to_dict_tracking_token()
    instance['time'] = end_time - start_time
    q[pos] = instance
    save_queries(q)
    print(instance['id'], instance['question'], instance["query_string"], instance['time'])

In [15]:
queries

[{'id': '1',
  'question': 'What is the area of Thailand?',
  'query_string': "SELECT area FROM country WHERE name = 'Thailand';",
  'type': 'simple',
  'token_usage': {'table_recall': {'total_tokens': 3784,
    'total_cost': 0.0072075,
    'prompt_tokens': 721,
    'completion_tokens': 3063},
   'column_recall': {'total_tokens': 5752,
    'total_cost': 0.011373499999999998,
    'prompt_tokens': 261,
    'completion_tokens': 5491},
   'sql_generation': {'total_tokens': 798,
    'total_cost': 0.001329,
    'prompt_tokens': 534,
    'completion_tokens': 264}}},
 {'id': '2',
  'question': 'What are the provinces with an area greater than 10000?',
  'query_string': 'SELECT province.name FROM province WHERE province.area > 10000;',
  'type': 'simple',
  'token_usage': {'table_recall': {'total_tokens': 2289,
    'total_cost': 0.0042144999999999995,
    'prompt_tokens': 727,
    'completion_tokens': 1562},
   'column_recall': {'total_tokens': 4013,
    'total_cost': 0.0079085,
    'prompt_tok